In [186]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod

In [187]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [188]:
W = 750
cost = [135, 139, 149, 150, 156, 163, 173,
        184, 192, 201, 210, 214, 221, 229, 240]
weight = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
opt = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
c = {}
w = {}
N = len(cost)
for i in range(N):
    c[i] = cost[i]
    w[i] = weight[i]

In [189]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [190]:
key1 = max(c, key=lambda k: c[k])
B = 1
A = B * c[key1] * 2

In [191]:
HA = Constraint(
    A * (W - sum(w[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(c[a] * x[a] for a in range(N))

In [192]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [193]:
dw_sampler = DWaveSampler(solver='Advantage_system4.1',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(Q, num_reads=10)

In [194]:
sampleset.record

rec.array([([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0], -2.69676266e+08, 1, 0.08),
           ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0], -2.67005066e+08, 1, 0.08),
           ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0], -2.68601239e+08, 1, 0.12),
           ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0], -2.69952919e+08, 1, 0.04),
           ([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0], -2.69028851e+08, 1, 0.12),
           ([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0], -2.68652759e+08, 1, 0.08),
           ([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0], -2.66368171e+08, 1, 0.16),
           ([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], -2.69809375e+08, 1, 0.04),
           ([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,

In [195]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [196]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]
        

In [197]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [198]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 286

broken
{'HA': (False, 48000.0)}
cost : 919
weight : 474


In [199]:
init = sampleset.samples()[-1]

In [200]:
t = (0, 5, 15, 20)
s = (1.0, 0.4, 0.4, 1.0)

schedule = list(zip(t, s))

In [201]:
dw_sampler = DWaveSampler(solver='Advantage_system4.1',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(
    Q, num_reads=10, anneal_schedule=schedule, initial_state=init)

In [202]:
sampleset.record

rec.array([([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1], -2.54275747e+08, 1, 0.12),
           ([1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0], -2.61764076e+08, 1, 0.04),
           ([0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1], -2.55468717e+08, 1, 0.  ),
           ([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0], -2.66450976e+08, 1, 0.08),
           ([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], -2.69789090e+08, 1, 0.04),
           ([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], -2.67781758e+08, 1, 0.08),
           ([1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], -2.63089635e+08, 1, 0.  ),
           ([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0], -2.63653445e+08, 1, 0.08),
           ([1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,

In [203]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [204]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [205]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [206]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 328

broken
{'HA': (False, 211680.0)}
cost : 770
weight : 401
